In [251]:
import pandas as pd
import plotly.express as px

In [300]:
df = pd.read_feather('sample.feather')

df.columns

Index(['index', 'CreationDate', 'Title', 'Tags', 'ViewCount', 'AnswerCount',
       'CommentCount', 'FavoriteCount'],
      dtype='object')

In [253]:
df.CreationDate.max()

Timestamp('2022-09-24 16:04:26.537000')

In [302]:
tags = (
    df.explode("Tags")
    .groupby([df.CreationDate.dt.to_period("M"), "Tags"])
    .agg({"ViewCount": "sum", "AnswerCount": "sum"})
    .reset_index()
)


In [303]:
tags

,CreationDate,Tags,ViewCount,AnswerCount
0,2018-01,additives,1126,0
1,2018-01,alfredo,3141,2
2,2018-01,artichokes,94,1
3,2018-01,asian-cuisine,3044,1
4,2018-01,baking,149677,28
...,...,...,...,...
10021,2022-09,waffle,36,0
10022,2022-09,whipped-cream,58,1
10023,2022-09,wok,84,1
10024,2022-09,yeast,188,2


In [304]:
tags.set_index('CreationDate', inplace=True)

In [305]:
rank = tags.loc[:'2022-06-30']

In [308]:
# list top tags
rank.groupby('Tags').sum().sort_values('ViewCount', ascending=False).head(10)

,ViewCount,AnswerCount
Tags,,
food-safety,1593445,1083
baking,1288023,1159
bread,825153,727
substitutions,712535,616
oven,591204,268
equipment,569575,587
chicken,474765,311
eggs,383875,311
meat,378411,317


In [309]:
mytags = ['baking','bread']

customrank = rank[rank.Tags.isin(mytags)]

customrank

,Tags,ViewCount,AnswerCount
CreationDate,,,
2018-01,baking,149677,28
2018-01,bread,55904,14
2018-02,baking,89965,21
2018-02,bread,591,3
2018-03,baking,25623,24
...,...,...,...
2022-04,baking,1754,13
2022-04,bread,1142,7
2022-05,baking,4195,10


In [310]:
# plot customrank
fig = px.line(
    customrank, x=customrank.index.to_timestamp(), y="ViewCount", color="Tags"
)
fig.show()


In [321]:
mytags = ['baking','bread']

customrank = rank[rank.Tags.isin(mytags)]

customrank

customrank.groupby('Tags').rolling(7, center=True).mean()

ViewCount  AnswerCount
Tags   CreationDate                           
baking 2018-01                NaN          NaN
       2018-02                NaN          NaN
       2018-03                NaN          NaN
       2018-04       54013.142857    18.428571
       2018-05       36700.857143    17.857143
...                           ...          ...
bread  2022-01        2577.714286     6.714286
       2022-02        2526.857143     6.571429
       2022-03                NaN          NaN
       2022-04                NaN          NaN
       2022-05                NaN          NaN

[107 rows x 2 columns]

In [319]:
fig = px.line(
    rollingavg, x=customrank.index.to_timestamp(), y="ViewCount",
    facet_col=rollingavg.index.get_level_values(0)
)
fig.show()
